## Projecting land and ocean sinks with carbon-carbon and carbon-climate feedback factors 

- This notebook loads carbon-carbon and carbon-climate feedback factors (Table 5.5, IPCC AR6 WG1), as well as temperature and atmospheric CO2 concentration scenarios\
       - reference: IPCC AR6 Ch5 (https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter05.pdf) and Arora et al. 2020 (https://bg.copernicus.org/articles/17/4173/2020/)
     - Note that CMIP6 Beta and Gamma factors were diagnosed from 1%/yr increasing CO2 runs to 4xCO2 (relative to preindustrial). This is most similar to the SSP585 scenario. 
     - For scenarios with slower increases or decreases in atmospheric CO2, results should become less accurate to what would occur in a full Earth System Model.
     - See McKinley et al. 2023 for ocean sink estimates from ESMs under a range of scenarios.

Student work to do
- Choose future atmospheric CO2/warming scenarios (part 4)
- Choose feedback factors (CMIP6 means used as default) (part 5)

Notebook returns
- Plots the cumulative ocean and land sinks
- Outputs sink magnitudes in 2100

For EESC4020 Humans and the Carbon Cycle, Columbia University, Fall 2024 \
    - Developed by G.A. McKinley and A. Shaum, Aug-Oct 2024

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget 
# ^ allows zooming in to the plots!

import cmocean as cm    

import gcsfs
fs = gcsfs.GCSFileSystem()

%run plotting_functions.ipynb

## 1. Load and print the IPCC Feedback factors (AR6 WG1 Table 5.5)

In [ ]:
ipcc_feedback = pd.read_csv('./data/ipcc_feedback_factors.csv')

In [ ]:
ipcc_feedback

In [ ]:
# Extract model mean values, used as default
cmip6_modelmean_beta_land = float(ipcc_feedback['β_Land (PgC ppm⁻¹)'][10][0:5])
cmip6_modelmean_gamma_land = float(ipcc_feedback['γ_Land (PgC K⁻¹)'][10][0:5])
cmip6_modelmean_beta_ocean = float(ipcc_feedback['β_Ocean (PgC ppm⁻¹)'][10][0:5])
cmip6_modelmean_gamma_ocean = float(ipcc_feedback['γ_Ocean (PgC K⁻¹)'][10][0:5])

## 2. Load, plot future Atmospheric CO2 Scenarios from IPCC

In [ ]:
ssp_xco2 = pd.read_csv('./data/scenario_atm_co2.csv')

# looking at 2015 through 2100:
ssp_xco2 = ssp_xco2.loc[:,'Scenario'].to_frame().join(ssp_xco2.loc[:,'2015':'2100'])

ssp_xco2.index = ssp_xco2['Scenario']
ssp_xco2 = ssp_xco2.drop(columns=['Scenario'])

scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp434-over','ssp585']
years = np.array(ssp_xco2.keys())

xco2_119 = ssp_xco2.loc[scenarios[0]]
xco2_126 = ssp_xco2.loc[scenarios[1]]
xco2_245 = ssp_xco2.loc[scenarios[2]]
xco2_370 = ssp_xco2.loc[scenarios[3]]
xco2_434 = ssp_xco2.loc[scenarios[4]]
xco2_585 = ssp_xco2.loc[scenarios[5]]

In [ ]:
#  For plotting to make cumulative starting in 2015, so subtracting the year 2015 values from each year
#xco2_119_start2015 = xco2_119.sub(xco2_119.iloc[0])
#xco2_126_start2015 = xco2_126.sub(xco2_126.iloc[0])
#xco2_245_start2015 = xco2_245.sub(xco2_245.iloc[0])
#xco2_370_start2015 = xco2_370.sub(xco2_370.iloc[0])
#xco2_434_start2015 = xco2_434.sub(xco2_434.iloc[0])
#xco2_585_start2015 = xco2_585.sub(xco2_585.iloc[0])

# For the calculation of the future impact of change in ocean sink, consider change from 2015
ssp_xco2_start2015 = ssp_xco2-ssp_xco2[['2015']].values

In [ ]:
plt.figure()
plt.xlabel('year')
plt.ylabel('Atmospheric CO$_2$ (ppm)')
plt.title('Projected Atmospheric CO$_2$ 2015-2100')

plt.xticks(ticks=[0,17,34,51,68,85],labels=years[::17]);

# for no,t_array in enumerate([xco2_119_start2015,xco2_126_start2015,xco2_245_start2015,
#                             xco2_370_start2015,xco2_434_start2015,xco2_585_start2015]):
for no,t_array in enumerate([xco2_119,xco2_126,xco2_245,
                             xco2_370,xco2_434,xco2_585]):
    plt.plot(years,t_array,label=scenarios[no])

plt.legend()

## 3. Load, plot future surface temperatures under these scenarios

In [ ]:
ssp_temps = pd.read_csv('./data/scenario_temperatures.csv')

# looking at 2015 through 2100:
ssp_temps = ssp_temps.loc[:,'Scenario'].to_frame().join(ssp_temps.loc[:,'2015':'2100'])

# to get the scenario name out of the data
ssp_temps.index = ssp_temps['Scenario']
ssp_temps = ssp_temps.drop(columns=['Scenario'])

years = np.array(ssp_temps.keys())

scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp434-over','ssp585']

temp119 = ssp_temps.loc[scenarios[0]]
temp126 = ssp_temps.loc[scenarios[1]]
temp245 = ssp_temps.loc[scenarios[2]]
temp370 = ssp_temps.loc[scenarios[3]]
temp434 = ssp_temps.loc[scenarios[4]]
temp585 = ssp_temps.loc[scenarios[5]]

In [ ]:
# to make cumulative starting in 2015, so subtracting the year 2015 values from each year
temp119_start2015 = temp119.sub(temp119.iloc[0])
temp126_start2015 = temp126.sub(temp126.iloc[0])
temp245_start2015 = temp245.sub(temp245.iloc[0])
temp370_start2015 = temp370.sub(temp370.iloc[0])
temp434_start2015 = temp434.sub(temp434.iloc[0])
temp585_start2015 = temp585.sub(temp585.iloc[0])

ssp_temps_start2015 = ssp_temps-ssp_temps[['2015']].values

In [ ]:
plt.figure()
plt.xlabel('year')
plt.ylabel('degrees kelvin/celsius')
plt.title('Global Mean Temperature Change from 2015-2100')

plt.xticks(ticks=[0,17,34,51,68,85],labels=years[::17]);

for no,t_array in enumerate([temp119_start2015,temp126_start2015,temp245_start2015,
                             temp370_start2015,temp434_start2015,temp585_start2015]):
    plt.plot(years,t_array,label=scenarios[no])

plt.legend()

## 4. Select Future Scenario

In [ ]:
# choices: scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp434-over','ssp585']
scenario_choice = 'ssp585'

In [ ]:
# select temperatures for selected scenario, starting in 2015
temp_scenario = ssp_temps_start2015.loc[scenario_choice]
# calculate delta xCO2 for selected scenario, relative to 2015
del_xco2_scenario = ssp_xco2_start2015.loc[scenario_choice]

## 5. Beta and Gamma values 

See directions in second cell to change Beta and Gamma from their default value

In [ ]:
# By default, takes CMIP6 mean values
beta_land = cmip6_modelmean_beta_land
gamma_land = cmip6_modelmean_gamma_land
beta_ocean = cmip6_modelmean_beta_ocean
gamma_ocean = cmip6_modelmean_gamma_ocean 

In [ ]:
# To change beta and gamma from defaults, remove the # (i.e. uncomment) and enter alternative value
# You can change one or more at a time. 
# If you with to return to default values, re-comment the line with #

#beta_land = 
#gamma_land = 
#beta_ocean = 
#gamma_ocean = 

In [ ]:
print('current beta land value:',beta_land)
print('current gamma land value:',gamma_land)
print('current beta ocean value:',beta_ocean)
print('current gamma ocean value:',gamma_ocean)

## 6. Calculate the future land and ocean sinks, relative to 2015, plot and print cumulative value at 2100

In [ ]:
# land calculations
del_carbon_land = beta_land * del_xco2_scenario + gamma_land * temp_scenario
del_carbon_land_carbononly = beta_land * del_xco2_scenario
del_carbon_land_temponly = gamma_land * temp_scenario

# ocean calculations
del_carbon_ocean = beta_ocean * del_xco2_scenario + gamma_ocean * temp_scenario
del_carbon_ocean_carbononly = beta_ocean * del_xco2_scenario
del_carbon_ocean_temponly =  gamma_ocean * temp_scenario

In [ ]:
f,ax = plt.subplots(1,2,figsize=(12,5),sharey=True)
f.suptitle(f'{scenario_choice}: land and ocean sinks')
    
ax[0].set_title(f'LAND: beta: {beta_land}, gamma: {gamma_land}',fontsize=10)
ax[1].set_title(f'OCEAN: beta: {beta_ocean}, gamma: {gamma_ocean}',fontsize=10)

ax[0].plot(del_carbon_land,c='red',label='Cumulative land sink from 2015')
ax[0].plot(del_carbon_land_carbononly,c='green',label='land sink (carbon)')
ax[0].plot(del_carbon_land_temponly,c='blue',label='land sink (temp)')

ax[1].plot(del_carbon_ocean,c='red',label='Cumulative ocean sink from 2015')
ax[1].plot(del_carbon_ocean_carbononly,c='green',label='ocean sink (carbon)')
ax[1].plot(del_carbon_ocean_temponly,c='blue', label='ocean sink (temp)')

ax[0].set_xticks(ticks=[0,17,34,51,68,85],labels=years[::17]);
ax[1].set_xticks(ticks=[0,17,34,51,68,85],labels=years[::17]);

ax[0].set_ylabel('Cumulative Carbon Sinks from 2015 (PgC)')

ax[0].margins(x=0)
ax[1].margins(x=0)

# grid:
ax[0].grid()
ax[1].grid()

# legend:
ax[0].legend()
ax[1].legend()

In [ ]:
print(f'Cumulative land sink in 2100: {del_carbon_land.iloc[-1]} PgC \n\
Cumulative ocean sink in 2100: {del_carbon_ocean.iloc[-1]} PgC \n\
Combined sink 2100: {del_carbon_ocean.iloc[-1] + del_carbon_land.iloc[-1]} PgC')